In [1]:
from models.cifar_resnet import cifar_resnet
import torch
from pprint import pprint

In [2]:
cresnet8 = cifar_resnet(variant=8).to(torch.device('cuda'))
cresnet8.eval()
print('Initialised resnet for evaluation')

Initialised resnet for evaluation


In [3]:
from fed_df_data_loader.get_crops_dataloader import get_distill_imgloader

distill_dataloader = get_distill_imgloader('./data/single_img_crops/crops', dataset_name='cifar10', batch_size=1024)

In [4]:
from strategy.clustering import cluster_embeddings

In [5]:
device = torch.device('cuda')
test_df,score = cluster_embeddings(dataloader=distill_dataloader, model=cresnet8, device=device, seed=42)

In [6]:
print(score)
test_df.head()

16809.14453125


,img,cluster,cluster_dist
img_no,,,
0,[[[[-2.390296 -2.4290657 -2.4290657 -2.429...,8,0.296791
1,[[[[ 1.4479177 1.5836121 0.8469853 2.126...,9,0.748865
2,[[[[-2.4290657 -2.4096808 1.4673026 1.622...,1,0.606561
3,[[[[-0.8395026 -0.95581204 0.24605286 1.215...,2,0.495888
4,[[[[ 1.5836121 1.370378 0.90514 0.827...,5,0.708190


In [7]:
from strategy.clustering import prune_clusters

In [8]:
test_df2 = prune_clusters(test_df,n_crops=3000,heuristic_percentage="60-40")
test_df2.head()


,img,cluster,cluster_dist
img_no,,,
0,[[[[-0.8395026 -1.0721215 -0.7813478 -0.645...,7,0.324818
1,[[[[-2.4290657 -2.0801373 0.24605286 0.846...,0,0.252581
2,[[[[-0.7813478 -0.47118917 -0.7038081 -1.130...,0,0.236755
3,[[[[-1.2659708 -0.72319305 -0.37426457 -0.471...,3,2.108626
4,[[[[ 2.514088 2.494703 2.4559333 2.320...,3,0.390215


In [9]:
from strategy.clustering import visualise_clusters

In [10]:
with open('test_img.png','wb') as f:
    visualise_clusters(test_df2,f,n_imgs=8,n_classes=10)

In [11]:
from strategy.clustering import prepare_for_transport

out = prepare_for_transport(test_df2)
print(type(out))

<class 'bytes'>


In [12]:
from strategy.clustering import extract_from_transport

img_loader = extract_from_transport(out)
imgs, labels = next(iter(img_loader))
print(imgs.size())
print(imgs[0].size())
print(labels[0])

torch.Size([512, 3, 32, 32])
torch.Size([3, 32, 32])
tensor(1)
